In [1]:
# Importación de librerías
import numpy as np  # Para operaciones numéricas y matrices
import matplotlib.pyplot as plt  # Para visualización gráfica
import matplotlib.animation as animation  # Para crear animaciones
from IPython.display import HTML  # Para mostrar animaciones en notebooks
import matplotlib.patches as patches  # Para dibujar formas (círculos)

class FSASimulationManualDistance:
    def __init__(self):

        # Parámetros de simulación propuestos
        self.n_centinelas = 30              # Número de centinelas
        self.n_aerodeslizadores = 4         # Número de aerodeslizadores
        self.area_size = 10                 # Tamaño del área de simulación (10x10) km
        self.min_separation = 1.5           # Separación inicial entre centinelas 1.5 km

        # Parámetros de optimizacion y simulacion
        self.centinela_visual_range = 0.5   # Rango visual de detección
        self.aerodeslizador_visual_range = 0.099  # Rango visual de detección
        self.step_size = 0.3                # Tamaño del paso de movimiento

        self.neutralization_threshold = 10  # Umbral para neutralización
        self.max_attempts = 1000            # Intentos máximos para colocar agentes
        self.mu = 0.3                       # Factor de hacinamiento para comportamiento grupal
        ####Su propósito es evitar la concentración excesiva de individuos en una misma zona,
        ####lo que podría reducir la eficiencia de la búsqueda.

        # Atributos para seguimiento de posiciones
        self.posiciones_visitadas = [[] for _ in range(self.n_centinelas)]  # Lista de arrays por centinela
        self.avoid_radius = 0.3  # Radio de evitación
        self.max_visited_positions = 50  # Máximo histórico a recordar por centinela

        # Inicialización
        self.reset_simulation()
        self.setup_visualization()

    def setup_visualization(self):
        """Configura la visualización con radios personalizados"""
        self.fig, self.ax = plt.subplots(figsize=(12, 10))
        # El hecho de ser de -1 al area +1 es para mejorar la visualizacion de escenario
        self.ax.set_xlim(-1, self.area_size + 1)
        self.ax.set_ylim(-1, self.area_size + 1)
        self.ax.set_title("Zona X", fontsize=30, fontweight='bold')
        self.ax.grid(True, alpha=0.3, linestyle='--')

        # Elementos gráficos principales
        self.centinela_plot = self.ax.scatter([], [], c='grey', s=100, label='Centinelas')
        self.aerodeslizador_plot = self.ax.scatter([], [], c='red', s=800, marker='^', label='Aerodeslizadores')
        self.senuelo_plot = self.ax.scatter([], [], c='orange', s=800, marker='^', label='Señuelo')

        # Círculos de radio para centinelas
        self.centinela_circles = [self.ax.add_patch(plt.Circle((0,0), self.centinela_visual_range,
                                    color='blue', fill=False, alpha=0.1, linewidth=0.2))
                                for _ in range(self.n_centinelas)]

        # Círculos de radio para aerodeslizadores
        self.aerodeslizador_circles = [self.ax.add_patch(plt.Circle((0,0), self.aerodeslizador_visual_range,
                                        color='pink', fill=False, alpha=0.2))
                                    for _ in range(self.n_aerodeslizadores)]

        # Contadores y marcadores
        self.contador_texts = [self.ax.text(0, 0, '', ha='center', va='center',
                                  fontsize=15, weight='bold') for _ in range(self.n_aerodeslizadores)]

        self.neutralized_markers = [self.ax.add_patch(plt.Circle((0,0), 0.3,
                                        color='black', fill=True, alpha=0.7))
                                  for _ in range(self.n_aerodeslizadores)]

        # Texto informativo
        self.info_text = self.ax.text(0.01, 0.009, '', transform=self.ax.transAxes,
                                    bbox=dict(facecolor='white', alpha=0.2))

        self.ax.legend(loc='upper right', fontsize=14)

    def update_visualization(self, iteration):
        """Actualiza la visualización"""
        # Actualizar posiciones principales
        self.centinela_plot.set_offsets(self.centinelas)

        # Actualizar círculos de centinelas
        for i, circle in enumerate(self.centinela_circles):
            circle.center = self.centinelas[i]

        # Separar aerodeslizadores activos y señuelo
        active_aero = []
        senuelo_pos = []

        for i in range(self.n_aerodeslizadores):
            pos = self.aerodeslizadores[i]

            # Actualizar círculos de aerodeslizadores
            self.aerodeslizador_circles[i].center = pos
            self.aerodeslizador_circles[i].set_visible(i not in self.neutralizados)

            if i not in self.neutralizados:
                if i == self.senuelo_idx:
                    senuelo_pos.append(pos)
                else:
                    active_aero.append(pos)

        # Actualizar plots principales
        self.aerodeslizador_plot.set_offsets(active_aero if active_aero else np.empty((0,2)))
        self.senuelo_plot.set_offsets(senuelo_pos if senuelo_pos else np.empty((0,2)))

        # Actualizar contadores y marcadores
        for i in range(self.n_aerodeslizadores):
            pos = self.aerodeslizadores[i]

            # Contadores
            self.contador_texts[i].set_position(pos + np.array([0, -0.5]))
            self.contador_texts[i].set_text(f"{self.contadores[i]}/{self.neutralization_threshold}")

            # Color del contador
            progress = min(1.0, self.contadores[i]/self.neutralization_threshold)
            color = (progress, 1-progress, 0)
            self.contador_texts[i].set_color(color)

            # Marcadores de neutralización
            self.neutralized_markers[i].center = pos
            self.neutralized_markers[i].set_visible(i in self.neutralizados)

        # Actualizar texto informativo
        info = f"Iteración: {iteration}\n"
        info += f"Aerodeslizadores neutralizados: {len(self.neutralizados)}/{self.n_aerodeslizadores}\n"
        info += f"Señuelo: Estado ->  {'neutralizado' if self.senuelo_idx in self.neutralizados else 'activo'}\n"
        # Configurar propiedades de texto
        font_properties = {
            'fontsize': 14,  # Tamaño aumentado (el normal suele ser 10-12)
            'fontweight': 'bold',  # Opcional: texto en negrita
            'fontfamily': 'sans-serif',  # Opcional: tipo de letra
            'color': 'darkblue'  # Opcional: color del texto
        }

        # Si el texto ya existe, actualizarlo
        if hasattr(self, 'info_text'):
            self.info_text.set_text(info)
            self.info_text.set_fontsize(14)  # Actualizar tamaño
        else:
            # Crear nuevo texto con tamaño grande
            self.info_text = self.ax.text(
                0.02,  # Posición x (coordenadas de ejes, 0-1)
                0.98,  # Posición y (cerca del borde superior)
                info,
                transform=self.ax.transAxes,
                verticalalignment='top',
                bbox=dict(facecolor='white', alpha=0.7, edgecolor='gray'),
                **font_properties  # Aplica las propiedades de fuente
            )

        return (self.centinela_plot, self.aerodeslizador_plot, self.senuelo_plot,
                *self.centinela_circles, *self.aerodeslizador_circles,
                *self.contador_texts, *self.neutralized_markers,
                self.info_text)

    def euclidean_distance(self, a, b):
        """Calcula distancia euclidiana entre puntos (ecuación 2)"""
        return np.linalg.norm(a - b)

    def generate_positions(self, n, min_dist, existing=None):
        """
        Genera posiciones aleatorias con distancia mínima
        Input
        n: número de posiciones a generar
        min_dist: distancia mínima requerida entre posiciones
        existing: posiciones existentes que deben ser evitadas (opcional)
        Output
        Convierte la lista de posiciones a un array de NumPy y lo retorna.
        """
        positions = []
        existing_list = existing.tolist() if existing is not None else [] #Convierte las posiciones existentes (si las hay) a una lista de Python

        for _ in range(n):
            attempts = 0 # Inicializa un contador de intentos para generar cada posición.
            while attempts < self.max_attempts:
                candidate = np.random.uniform(0, self.area_size, 2) # Genera una posición candidata aleatoria
                #Usa una comprensión de generador para calcular todas las distancias
                if len(existing_list) == 0 or min(self.euclidean_distance(candidate, p) for p in existing_list) >= min_dist:
                    positions.append(candidate)
                    existing_list.append(candidate)
                    break
                attempts += 1
            else:
                raise ValueError(f"No se pudo generar posición después de {self.max_attempts} intentos")
        return np.array(positions)

    def reset_simulation(self):
        """Reinicia la simulación"""
        self.centinelas = self.generate_positions(self.n_centinelas, self.min_separation) # Condiciona las posiciones con una distancia
        self.aerodeslizadores = np.random.uniform(0, self.area_size, (self.n_aerodeslizadores, 2)) # Genera posiciones aleatoriamente

        # Configuración de calidad
        self.calidad_aero = np.ones(self.n_aerodeslizadores)
        self.senuelo_idx = np.random.randint(0, self.n_aerodeslizadores)
        self.calidad_aero[self.senuelo_idx] = 0.3  # Señuelo

        # Estado
        self.neutralizados = []
        self.contadores = np.zeros(self.n_aerodeslizadores, dtype=int)

        # Históricos
        self.historial_contadores = []
        self.historial_posiciones = []

        # Reiniciar posiciones visitadas
        self.posiciones_visitadas = [[] for _ in range(self.n_centinelas)]

    def get_neighbors(self, position, points, radius):
        """
        Encuentra vecinos dentro de un radio
        Input
        position: Coordenadas (x, y) del punto central desde donde se buscarán vecinos.
        points: Lista/array de todos los puntos posibles
        radius: Radio máximo para considerar un punto como "vecino".

        Output
        Lista de índices de los puntos dentro del radio especificado.
        """
        return [i for i, p in enumerate(points) if self.euclidean_distance(position, p) <= radius]

    def update_counters(self):
        """Actualiza contadores manualmente"""
        self.contadores.fill(0)

        for i in range(self.n_aerodeslizadores):
            if i not in self.neutralizados:
                # Buscar centinelas dentro del radio visual (ecuación 2)
                for j in range(self.n_centinelas):
                    if self.euclidean_distance(self.aerodeslizadores[i], self.centinelas[j]) <= self.aerodeslizador_visual_range * 1.5:
                        self.contadores[i] += 1

        self.historial_contadores.append(self.contadores.copy())

    def record_position(self, idx_centinela, position):
        """
        Guarda la posición actual, limitando el historial
        Input
        idx_centinela: Índice o identificador del centinela cuya posición se va a registrar.
        position: Coordenadas (x, y) de la posición actual que se desea guardar.


        """
        self.posiciones_visitadas[idx_centinela].append(position.copy())
        # Si el historial excede el límite, se elimina la posición más antigua (Evitar mal uso de la memoria).
        if len(self.posiciones_visitadas[idx_centinela]) > self.max_visited_positions:
            self.posiciones_visitadas[idx_centinela].pop(0)

    def calculate_movement(self, idx_centinela):
        """
        Implementa el comportamiento de búsqueda/presa del FSA

        Este algoritmo entrena a cada pez artificial (AF)
        enseñando cuatro tipos de comportamiento básicos: búsqueda o presa,
        agrupamiento o enjambre, seguimiento y aleatorio

        Input
        idx_centinela: Índice o identificador del centinela cuya posición se va a registrar.

        """
        # Obtiene la posición actual del centinela.
        pos = self.centinelas[idx_centinela]

        def is_too_close(new_pos):
            """Verifica si la posición está cerca de alguna visitada"""
            for visited in self.posiciones_visitadas[idx_centinela]:
                if self.euclidean_distance(new_pos, visited) < self.avoid_radius:
                    return True
            return False

        # 1. Comportamiento de de busqueda o presa
        ## lista de aerodeslizadores no neutralizados
        targets = [i for i in range(self.n_aerodeslizadores) if i not in self.neutralizados]

        if not targets:
            return pos  # No hay objetivos

        # Calcula distancias a los objetivos y verifica cuáles están dentro del rango visual.
        distancias = [self.euclidean_distance(pos, self.aerodeslizadores[i]) for i in targets]
        en_rango = [d <= self.aerodeslizador_visual_range for d in distancias]

        # Si hay objetivos en rango
        if any(en_rango):
            fitness = []
            for i, idx in enumerate(targets):
                if en_rango[i]:
                    # Fórmula de congestión
                    crowding = 1 / (1 + self.contadores[idx]/self.neutralization_threshold)
                    fitness.append(self.calidad_aero[idx] * crowding / (distancias[i] + 1e-6))
                else:
                    fitness.append(-1)

            # Selecciona el objetivo con mayor fitness.
            mejor_idx = targets[np.argmax(fitness)]
            target = self.aerodeslizadores[mejor_idx]

            # Mover hacia el objetivo con paso aleatorio
            direction = target - pos
            distance = self.euclidean_distance(target, pos)
            if distance > 0:
                step = min(self.step_size, distance)
                random_factor = np.random.uniform(0, 1)
                # np.clip asegura que la posición permanezca dentro del área.
                new_pos = np.clip(pos + (direction/distance)*step*random_factor, 0, self.area_size)

                if not is_too_close(new_pos):
                    self.record_position(idx_centinela, new_pos)
                    return new_pos

        # 2. Comportamiento de agrupacion o enjambre
        vecinos = self.get_neighbors(pos, self.centinelas, self.centinela_visual_range)
        vecinos = [i for i in vecinos if i != idx_centinela]

        if vecinos:
            # Calcular centro del enjambre
            X_centro = np.mean([self.centinelas[i] for i in vecinos], axis=0)
            ny = len(vecinos)

            # Condición de hacinamiento
            if (ny/self.n_centinelas) > self.mu:
                # Mover hacia el centro
                direction = X_centro - pos
                distance = self.euclidean_distance(X_centro, pos)
                if distance > 0:
                    random_factor = np.random.uniform(0, 1)
                    new_pos = np.clip(pos + (direction/distance)*self.step_size*random_factor, 0, self.area_size)

                    if not is_too_close(new_pos):
                        self.record_position(idx_centinela, new_pos)
                        return new_pos

        # 3. Movimiento exploratorio
        closest_idx = targets[np.argmin(distancias)]
        direction = self.aerodeslizadores[closest_idx] - pos
        distance = self.euclidean_distance(self.aerodeslizadores[closest_idx], pos)

        if distance > 0:
            # Intentar varias direcciones aleatorias para evitar posiciones visitadas
            for _ in range(10):
                random_dir = np.random.uniform(-0.5, 0.5, 2)
                candidate_dir = (direction/distance)*0.7 + random_dir*0.3
                new_pos = np.clip(pos + candidate_dir*self.step_size*0.5, 0, self.area_size)

                if not is_too_close(new_pos):
                    self.record_position(idx_centinela, new_pos)
                    return new_pos

            # Si no encuentra posición nueva, usar la última calculada
            new_pos = np.clip(pos + (direction/distance)*self.step_size*0.5, 0, self.area_size)
            self.record_position(idx_centinela, new_pos)
            return new_pos

        return pos

    def check_neutralization(self):
        """
        Verifica neutralización de aerodeslizadores y detiene la simulación cuando todos están neutralizados.
        Muestra un mensaje gráfico cuando se completa la neutralización.
        """
        for i in range(self.n_aerodeslizadores):
            if i not in self.neutralizados and self.contadores[i] >= self.neutralization_threshold:
                self.neutralizados.append(i)

                # Dispersar centinelas cercanos
                nearby = self.get_neighbors(self.aerodeslizadores[i], self.centinelas, self.aerodeslizador_visual_range*1.8)

                for idx in nearby:
                    direction = self.centinelas[idx] - self.aerodeslizadores[i]
                    distance = self.euclidean_distance(self.centinelas[idx], self.aerodeslizadores[i])
                    if distance > 0:
                        self.centinelas[idx] += (direction/distance)*self.step_size*2
                        self.centinelas[idx] = np.clip(self.centinelas[idx], 0, self.area_size)

        # Verificar si todos han sido neutralizados
        if len(self.neutralizados) == self.n_aerodeslizadores:
            self.ax.text(0.5, 0.5, '¡Misión completada!\nTodos neutralizados',
                        fontsize=24, color='red', weight='bold',
                        ha='center', va='center', transform=self.ax.transAxes,
                        bbox=dict(facecolor='white', alpha=0.8, edgecolor='black'))

            # Dibujar el mensaje inmediatamente
            plt.draw()
            plt.pause(0.1)

            # Detener la simulación
            self.simulation_running = False  # Asegúrate de tener este flag en tu clase
            return True  # Indica que la simulación debe detenerse

        return False

    def step(self, iteration):
        """Ejecuta un paso de simulación"""
        self.update_counters()

        # Mover centinelas
        new_positions = np.array([self.calculate_movement(i) for i in range(self.n_centinelas)])
        self.centinelas = np.clip(new_positions, 0, self.area_size)

        # Verificar neutralización
        self.check_neutralization()

        # Actualizar visualización
        return self.update_visualization(iteration)

    def run_simulation(self, n_iter=100):
        """Ejecuta la simulación completa"""
        def animate(i):
            return self.step(i)

        anim = animation.FuncAnimation(self.fig, animate, frames=n_iter,
                                     init_func=lambda: self.update_visualization(0),
                                     blit=False, interval=200)

        plt.close()
        return anim

# Ejecutar simulación
sim = FSASimulationManualDistance()
anim = sim.run_simulation(n_iter=100)
# HTML(anim.to_jshtml())
HTML(anim.to_html5_video())

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

...

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>